<a href="https://colab.research.google.com/github/isaac-mackey/mind-uploading/blob/main/Journal_Labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and configuration

In [ ]:
%%capture
from google.colab import drive
drive.mount('/content/drive')
# drive.mount('/content/drive', force_remount=True)

!pip install openai
from openai import OpenAI
import json


def show_json(obj):
    display(json.loads(obj.model_dump_json()))

## Load entries and label-to-date dictionary

In [ ]:
import re

def read_journal_entries(file_path):
    with open(file_path, 'r') as file:
        entries = []
        current_entry = None

        for line in file:
            # Check if the line matches the pattern for a date entry
            if re.match(r'^\d{4}/\d{2}/\d{2}', line.strip()):
                # If there's a previous entry, add it to the list
                if current_entry:
                    entries.append(current_entry)
                # Start a new entry
                current_entry = {'date': line.strip()}
                current_entry['content'] = ""
            elif current_entry:
                # If it's not a date line and there's a current entry being processed, add the line to the content
                current_entry['content'] += line

        # Add the last entry after the loop
        if current_entry:
            entries.append(current_entry)

    return entries

# Replace 'file_path' with the path to your text file
# file_path = '/content/drive/My Drive/march-2011-until-jan-2018-journal.txt'
input_file_path = '/content/drive/My Drive/isaac_mackey@ucsb.edu 2023-11-13 17:45/Personal Journaling/full_journal_march_2011_to_march_2024.txt'

entries = read_journal_entries(input_file_path)

for year in [str(2011+x) for x in range(15)]:
    num_entries_for_year = [e['date'][:4] for e in entries].count(year)
    print(year+':',str(num_entries_for_year))

# Print the dictionaries for each entry
for entry in entries[:3]:
    print(entry)
    print("="*50)

for entry in entries[-3:]:
    print(entry)
    print("="*50)

entries_date_to_text = {}
for item in entries:
    yyyymmdd = item['date'][:10]
    entry_text = item['content']
    entries_date_to_text[yyyymmdd] = entry_text

i = 0
for k,v in entries_date_to_text.items():
    if i == 2:
        break
    i += 1
    print(k)
    print(v)
    print()

with open('/content/drive/My Drive/isaac_mackey@ucsb.edu 2023-11-13 17:45/Personal Journaling/label-to-dates-json-gpt3-journal-1_2024_04_07_07:57:46.json', 'r') as f:
  label_to_dates_dict = json.load(f)
label_to_dates_dict = {x['label']:x['dates'] for x in label_to_dates_dict}
label_to_dates_dict['Lost wallet']

## Upload files for retrieval

In [ ]:
import os
for x in os.listdir('/content/drive/My Drive/Personal Journaling'):
  print(x)
  if "personal" in x:
    print(x)
  if 'txt-' in x:
    print(x)
    # pass

In [ ]:
input_file_path = '/content/drive/My Drive/sms.json'

# Replace with your actual file name
with open(input_file_path, 'r') as file:
    lines = file.readlines()

output_file_path = input_file_path + '.jsonl'

with open(output_file_path, 'w') as jsonl_file:
    for line in lines:
        # Assuming each line is a simple string
        # Convert it into a JSON object with a specific key
        json_obj = json.dumps({"data": line.strip()})
        jsonl_file.write(json_obj + '\n')

In [ ]:
# For Fine-Tune
# data_file = '/content/drive/My Drive/sms-20231224020008.xml-role-system-user-5.json.jsonl'

# For Assistants (Retrieval)
data_file = '/content/drive/My Drive/Personal Journaling/full_journal.txt'

response = client.files.create(
  file=open(data_file, "rb"),
  # purpose="fine-tune"
  purpose="assistants"
)

In [ ]:
show_json(response)

## Load journal entries

In [ ]:
import re

def read_journal_entries(file_path):
    with open(file_path, 'r') as file:
        entries = []
        current_entry = None

        for line in file:
            # Check if the line matches the pattern for a date entry
            if re.match(r'^\d{4}/\d{2}/\d{2}', line.strip()):
                # If there's a previous entry, add it to the list
                if current_entry:
                    entries.append(current_entry)
                # Start a new entry
                current_entry = {'date': line.strip()}
                current_entry['content'] = ""
            elif current_entry:
                # If it's not a date line and there's a current entry being processed, add the line to the content
                current_entry['content'] += line

        # Add the last entry after the loop
        if current_entry:
            entries.append(current_entry)

    return entries

# Replace 'file_path' with the path to your text file
# file_path = '/content/drive/My Drive/march-2011-until-jan-2018-journal.txt'
input_file_path = '/content/drive/My Drive/isaac_mackey@ucsb.edu 2023-11-13 17:45/Personal Journaling/full_journal_march_2011_to_march_2024.txt'

entries = read_journal_entries(input_file_path)

for year in [str(2011+x) for x in range(15)]:
    num_entries_for_year = [e['date'][:4] for e in entries].count(year)
    print(year+':',str(num_entries_for_year))

# Print the dictionaries for each entry
for entry in entries[:3]:
    print(entry)
    print("="*50)

for entry in entries[-3:]:
    print(entry)
    print("="*50)

In [ ]:
entries_date_to_text = {}
for item in entries:
    yyyymmdd = item['date'][:10]
    entry_text = item['content']
    entries_date_to_text[yyyymmdd] = entry_text

In [ ]:
i = 0
for k,v in entries_date_to_text.items():
    if i == 2:
        break
    i += 1
    print(k)
    print(v)
    print()

## Journal entry topic labeling with function tools

In [ ]:
topics = ['childhood', 'family', 'academics', 'education', 'goals', 'health', 'exercise', 'hobbies', 'travel', 'friends', 'morals', 'career', 'relationships', 'technology', 'art', 'culture', 'finance', 'environment', 'sports', 'food', 'entertainment']

longer_topics = [
    'childhood', 'family', 'academics', 'education', 'goals', 'health', 'exercise', 'hobbies', 'travel',
    'friends', 'morals', 'career', 'relationships', 'technology', 'art', 'culture', 'finance', 'environment',
    'sports', 'food', 'entertainment', 'routines',
    'passion', 'emotions', 'intelligence', 'core values', 'identity', 'challenges', 'achievements',
    'inspirations', 'reflections', 'ambitions', 'social',
    'communication', 'humor', 'culture',
    'conflicts', 'comfort', 'community', 'tradition',
    'language', 'leadership', 'decisions', 'future',
    'social media', 'sleep', 'reading', 'music'
]

system_message = "Tell me information like keywords of relevance about some diary entries."

my_assistant = client.beta.assistants.create(
    instructions=system_message,
    name="Diary Labeling Assistant",
    tools=[#{"type": "retrieval"},
           {
      "type": "function",
      "function": {
          "name": "getRelevantTopics",
          "description": "Return the topics are relevant to a diary entry",
          "parameters": {
              "type": "object",
              "properties": {
                  "entry": {"type": "string"},
                  "keywords": {"type": "array", "items": {"type": "string"}, "description": "Keywords to determine relevance"}
                            },
              "required": ["entry", "keywords"]
                        }
                  }
          },
           ],
    model='gpt-3.5-turbo-1106',
    #model="gpt-4-turbo-preview",
    file_ids= [] #[files_to_retrieve]
)

### Example function calling

In [ ]:
from openai import OpenAI
import json

client = OpenAI(
)

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
print(run_conversation())

ChatCompletion(id='chatcmpl-96fMIZV2SrkQsQUh8qDuiP7QLSGCB', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current weather in:\n\n- San Francisco: 72°C\n- Tokyo: 10°C\n- Paris: 22°C', role='assistant', function_call=None, tool_calls=None))], created=1711375978, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_3bc1b5746c', usage=CompletionUsage(completion_tokens=26, prompt_tokens=169, total_tokens=195))


### Actual label function calling

In [ ]:
import time
import random

def wait_on_run(run, thread):
    print("wait on run")
    # show_json(run)
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

def wait_on_run_action(run, thread):
    print("wait on run action")
    show_json(run)
    while run.status == "queued" or run.status == "in_progress": # or run.status == "requires_action":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        print("wait on run action loop")
        show_json(run)
        time.sleep(0.5)
    return run

def callTools(tool_calls):
# the parameter comes straight from the openAI run
    tool_outputs = []
    for t in tool_calls:
        functionName = t.function.name
        attributes = json.loads(t.function.arguments)
        try:
            functionResponse =globals()[functionName](attributes)
        except:
             # we just tell openAi we couldn't :)
            functionResponse = { "status" : 'Error in function call '+functionName+'('+t.function.arguments+')' }
        tool_outputs.append(  { "tool_call_id": t.id , "output": json.dumps(functionResponse) })
    return tool_outputs

entry_labels = []

i = 0

for entry in entries:
    print('i:',str(i),'out of',str(len(entries)))
    i += 1
    date = entry['date']
    content = entry['content']

    thread = client.beta.threads.create()
    # show_json(thread)

    content = "From the following topics: "+str(longer_topics)+", list the topics related to the following entry: "+content

    # print(content)

    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=content,
        file_ids=[] #files_to_retrieve
    )
    # show_json(message)

    run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=my_assistant.id
    )

    run = wait_on_run(run, thread)

    if run.status == 'completed' or run.status == 'requires_action':
      messages = client.beta.threads.messages.list(
        thread_id=thread.id
      )
      # show_json(messages)
      m = []
      for y in messages:
          # print(y)
          m.append(y)
      m.reverse()

      # print('keywords')
      keywords = json.loads(run.required_action.submit_tool_outputs.tool_calls[0].function.arguments)['keywords']
      # print(keywords)

      # print(m)
      # print(m[0])

      answer = ""
      for x in m:
          if x.assistant_id:
              answer += ' '+x.content[0].text.value
      entry_labels.append((date, keywords))
    else:
      print("run.status")
      print(run.status)

In [ ]:
for x in entry_labels:
    print(x[0])
    print(str(x[1]))

In [ ]:
json.loads(action_run.required_action.submit_tool_outputs.tool_calls[0].function.arguments)['keywords']

In [ ]:
functionName = t.function.name
attributes = json.loads(t.function.arguments)
try:
    functionResponse =globals()[functionName](attributes)
except:
      # we just tell openAi we couldn't :)
    functionResponse = { "status" : 'Error in function call '+functionName+'('+t.function.arguments+')' }
tool_outputs.append(  { "tool_call_id": t.id , "output": json.dumps(functionResponse) })

## Journal entry topic labeling without function tools

In [ ]:
topics = ['childhood', 'family', 'academics', 'education', 'goals', 'health', 'exercise', 'hobbies', 'travel', 'friends', 'morals', 'career', 'relationships', 'technology', 'art', 'culture', 'finance', 'environment', 'sports', 'food', 'entertainment',
          'military', 'swimming', 'cycling', 'wrestling', 'physics']

longer_topics = [
    'childhood', 'family', 'academics', 'education', 'goals', 'health', 'exercise', 'hobbies', 'travel',
    'friends', 'morals', 'career', 'relationships', 'technology', 'art', 'culture', 'finance', 'environment',
    'sports', 'food', 'entertainment', 'routines',
    'passion', 'emotions', 'intelligence', 'core values', 'identity', 'challenges', 'achievements',
    'inspirations', 'reflections', 'ambitions', 'social',
    'communication', 'humor', 'culture',
    'conflicts', 'comfort', 'community', 'tradition',
    'language', 'leadership', 'decisions', 'future',
    'social media', 'sleep', 'reading', 'music'
]

pure_system_message = "You are an assistant that identifies and extracts keywords of relevance from a person's diary entries."

my_assistant = client.beta.assistants.create(
    instructions=pure_system_message,
    name="Diary Entry Labeling Assistant",
    tools=[],
    model='gpt-3.5-turbo-1106',
    #model="gpt-4-turbo-preview",
    file_ids= [] #[files_to_retrieve]
)

In [ ]:
import time
import random

def wait_on_run(run, thread):
    print("wait on run")
    # show_json(run)
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

created_entry_labels = []

i = 0

for entry in entries:
    print('i:',str(i),'out of',str(len(entries)))
    i += 1
    date = entry['date']
    content = entry['content']

    thread = client.beta.threads.create()
    # show_json(thread)

    content = "Return a Python list of keywords that could be used to identify or classify the following entry: "+content

    # print(content)

    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=content,
        file_ids=[] #files_to_retrieve
    )
    # show_json(message)

    run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=my_assistant.id
    )

    run = wait_on_run(run, thread)

    if run.status == 'completed' or run.status == 'requires_action':
      messages = client.beta.threads.messages.list(
        thread_id=thread.id
      )
      # show_json(messages)
      m = []
      for y in messages:
          # print(y)
          m.append(y)
      m.reverse()

      answer = ""
      for x in m:
          if x.assistant_id:
              answer += ' '+x.content[0].text.value
      created_entry_labels.append((date, answer))
    else:
      print("run.status")
      print(run.status)

In [ ]:
for x in created_entry_labels[100:104]:
  print(x[0])
  print(x[1])
  print()

In [ ]:
import re

def extract_keywords_numbered(diary_entry):
    # Use regular expression to find the lines containing keywords
    keyword_lines = re.findall(r'\d+\.\s+(.*)', diary_entry)

    # Extract keywords from the lines
    keywords = [line.strip() for line in keyword_lines]

    return keywords

def extract_keywords_dashed(diary_entry):
    # Use regular expression to find the lines containing keywords
    keyword_lines = re.findall(r'-\s*(.*)', diary_entry)

    # Extract keywords from the lines
    keywords = [line.strip() for line in keyword_lines]

    return keywords

# Example usage
diary_entry = """2011/07/01 (Friday)
Based on the provided diary entry, the following keywords could be used to identify or classify the entry:

1. Cycling
2. Adventure
3. Railroad tracks
4. Mile-long straight aways
5. Bonding
6. Gas station
7. Parenting
8. Rules
9. Mistakes
10. Disappearance
11. Responsibility
12. Death"""

keywords = extract_keywords_numbered(diary_entry)
print(keywords)

In [ ]:
created_entry_label_dict = {}
i = 0
j = 0
jj = 0
k = 0
for x,y in created_entry_labels:
    if x not in created_entry_label_dict:
        try:
            my_list = eval(y)
            if type(my_list) == type([]):
                created_entry_label_dict[x] = my_list
                i += 1
        except SyntaxError:
            y = y.replace("```python", "").replace("```", "").strip()
            try:
                my_list = eval(y)
                if type(my_list) == type([]):
                    created_entry_label_dict[x] = my_list
                    j += 1
            except Exception as e:
                jj += 1
        except Exception as e:
            k += 1
print("i",str(i))
print("j",str(j))
print("jj",str(jj))
print("k",str(k))

In [ ]:
created_entry_label_dict = {}
i = 0
ii = 0
j = 0
jj = 0
k = 0
for x,y in created_entry_labels:
    if x not in created_entry_label_dict:
        try:
            my_list = extract_keywords_numbered(y)
            if type(my_list) == type([]) and len(my_list) > 0:
                created_entry_label_dict[x] = my_list
                i += 1
            else:
                my_list = extract_keywords_dashed(y)
                if type(my_list) == type([]) and len(my_list) > 0:
                    created_entry_label_dict[x] = my_list
                    ii += 1
        except SyntaxError:
            y = y.replace("```python", "").replace("```", "").strip()
            try:
                my_list = eval(y)
                if type(my_list) == type([]):
                    created_entry_label_dict[x] = my_list
                    j += 1
            except Exception as e:
                jj += 1
        except Exception as e:
            k += 1
print("i",str(i))
print("ii",str(ii))
print("j",str(j))
print("jj",str(jj))
print("k",str(k))

In [ ]:
i = 0
for x,y in created_entry_label_dict.items():
    print(x)
    print(y)
    print()
    i += 1
    if i == 20:
        break

In [ ]:
from collections import Counter

# Sample dictionary a and list b
a = created_entry_label_dict

# Flatten the values in dictionary a into a single list
all_labels = [item for sublist in a.values() for item in sublist]

# Count occurrences of each value in b within all_values
counts = Counter(all_labels)

all_labels = list(set(all_labels))

label_to_date_dict = {item: [] for item in all_labels}

for date,label_list in created_entry_label_dict.items():
    for label in label_list:
        # Append the key to the list of keys for the corresponding value in the mapping dictionary
        label_to_date_dict[label].append(date)

i = 0
for label,dates in label_to_date_dict.items():
    print(label)
    print(dates)
    print()
    i += 1
    if i == 5:
        break

# Output the counts
for item in all_labels:
    if counts[item] > 10:
        print(f"{item}: {counts[item]}")

## Save Label-to-Dates-Dict

In [ ]:
import json

label_to_dates_json_list = []

for label,dates in label_to_date_dict.items():
    label_to_dates_json_list.append({"label":label,"dates":dates})

json_string = json.dumps(label_to_dates_json_list)

conversation_name = "label-to-dates-json-gpt3-journal-1"

print("Conversation name:",conversation_name)

conversation_name_check = input("Confirm conversation name (Y) or provide a name for the conversation: ")
# Validate the conversation name

if not conversation_name_check.lower() == "y":
    conversation_name = conversation_name_check

while " " in conversation_name:
  print("Invalid conversation name. Please enter a name without spaces.")
  conversation_name = input("Provide a name for the conversation: ")

import datetime
eastern_now_str = datetime.datetime.utcnow().astimezone(datetime.timezone(datetime.timedelta(hours=-4))).strftime("%Y_%m_%d_%H:%M:%S")
conversation_date = eastern_now_str

# Create the file name.
file_name = f"{conversation_name}_{conversation_date}"

print('json file_name',file_name)

# Save the JSON string to a file in Google Drive.
with open(f"/content/drive/My Drive/isaac_mackey@ucsb.edu 2023-11-13 17:45/Personal Journaling//{file_name}.json", "w") as f:
    f.write(json_string)

Conversation name: label-to-dates-json-gpt3-journal-1
Confirm conversation name (Y) or provide a name for the conversation: y


## Save date-to-labels dict

In [ ]:
conversation_name = "date-to-labels-json-gpt3-journal-1"

print("Conversation name:",conversation_name)

conversation_name_check = input("Confirm conversation name (Y) or provide a name for the conversation: ")
# Validate the conversation name

if not conversation_name_check.lower() == "y":
    conversation_name = conversation_name_check

while " " in conversation_name:
  print("Invalid conversation name. Please enter a name without spaces.")
  conversation_name = input("Provide a name for the conversation: ")

import datetime
eastern_now_str = datetime.datetime.utcnow().astimezone(datetime.timezone(datetime.timedelta(hours=-4))).strftime("%Y_%m_%d_%H:%M:%S")
conversation_date = eastern_now_str

# Create the file name.
file_name = f"{conversation_name}_{conversation_date}"

print('json file_name',file_name)

with open(f'/content/drive/My Drive/isaac_mackey@ucsb.edu 2023-11-13 17:45/Personal Journaling//{file_name}.json', 'w') as f:
  json.dump(created_entry_label_dict, f)

Conversation name: date-to-labels-json-gpt3-journal-1
Confirm conversation name (Y) or provide a name for the conversation: y


## Load label_to_dates dict

In [ ]:
with open('/content/drive/My Drive/isaac_mackey@ucsb.edu 2023-11-13 17:45/Personal Journaling/label-to-dates-json-gpt3-journal-1_2024_04_07_07:57:46.json', 'r') as f:
  label_to_dates_dict = json.load(f)
label_to_dates_dict = {x['label']:x['dates'] for x in label_to_dates_dict}
label_to_dates_dict['Lost wallet']

## Print and search labels

In [ ]:
# prompt: print the top k labels with the most dates in their entry in label_to_date_dict

# First, we need to count the number of dates associated with each label.
label_counts = {}
for label, dates in label_to_dates_dict.items():
    label_counts[label] = len(dates)

# Then, we can sort the labels by their counts in descending order.
sorted_labels = sorted(label_counts, key=label_counts.get, reverse=True)

# Finally, we can print the top k labels with the most dates.
k = 20  # Change this value to adjust the number of labels to print.
for label in sorted_labels[:k]:
    print(f"Label: {label}, Dates: {label_counts[label]}")

In [ ]:
# prompt: Write a user input query to get a few keyword successively and print the text of the journal entris that are labeled with any of those keywords

import random

keyword = "start"
keyword_list = []

top_k = 10
random_k = 10

for label in random.sample(sorted_labels[:top_k*3], top_k) + random.sample(sorted_labels, random_k):
    print(f"Label: {label}, Dates: {label_counts[label]}")

while keyword != "":
    print("\nEnter a keyword to get the related journal entries: (enter blank to stop)")
    keyword = input()
    try:
        print("You entered: ", keyword,',',str(len(label_to_dates_dict[keyword])),'entries')
    except KeyError:
        print("You entered: ", keyword,', 0 entries')
    keyword_list.append(keyword)

dates_selected = []

for keyword in keyword_list:
    if keyword in label_to_dates_dict:
        for date in label_to_dates_dict[keyword]:
            dates_selected.append(date[:10])

dates_selected = list(set(dates_selected))

window_size = 80

if not dates_selected:
    print("No entries found for "+' '.join(keyword_list))
else:
    for date in list(set(dates_selected)):
        print("Date:", date)
        print()
        entry = entries_date_to_text[date]
        lines = entry.split("\n")
        for line in lines:
          if line == "":
            # print()  # Print a blank line
            continue
          for i in range(0, len(line), window_size):
            print(line[i:i + window_size])
          print()  # Print a newline after each block
        print('#'*window_size)
        print()